<a href="https://colab.research.google.com/github/esgryaznova/compling_nlp_hse_course/blob/master/homework_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [ ]:
!pip install tokenizers matplotlib sklearn

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
de_sents = open('opus.de-en-train.de.txt').read().splitlines()
en_sents = open('opus.de-en-train.en.txt').read().splitlines()

In [4]:
de_sents[-1], en_sents[-1]

('Es geht nicht darum, hier zu kriminalisieren, zu emotionalisieren, sondern wir müssen uns mit dem Thema auseinandersetzen.',
 'I am not talking about criminalising it or reacting to it emotionally, but we must tackle this subject.')

In [5]:
tokenizer_de = Tokenizer(BPE())
tokenizer_de.pre_tokenizer = Whitespace()
trainer_de = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_de.train(files=["opus.de-en-train.de.txt"], trainer=trainer_de)

tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.de-en-train.en.txt"], trainer=trainer_en)

In [6]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
tokenizer_de.save('tokenizer_de')
tokenizer_en.save('tokenizer_en')

In [7]:
tokenizer_de = Tokenizer.from_file("tokenizer_de")
tokenizer_en = Tokenizer.from_file("tokenizer_en")

In [8]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [9]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_en.token_to_id('[PAD]')
PAD_IDX

3

In [10]:
# ограничимся длинной в 30 и 35 (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_de, max_len_en = 30, 35

In [11]:
X_de = [encode(t, tokenizer_de, max_len_de) for t in de_sents]
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]

In [12]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_de, texts_en):
        self.texts_de = [torch.LongTensor(sent) for sent in texts_de]
        self.texts_de = torch.nn.utils.rnn.pad_sequence(self.texts_de, padding_value=PAD_IDX)
        
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)

        self.length = len(texts_de)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_de = self.texts_de[:, index]
        ids_en = self.texts_en[:, index]

        return ids_de, ids_en

In [13]:
X_de_train, X_de_valid, X_en_train, X_en_valid = train_test_split(X_de, X_en, test_size=0.05)

In [14]:
training_set = Dataset(X_de_train, X_en_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [15]:
valid_set = Dataset(X_de_valid, X_en_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

In [16]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [17]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_de, texts_en) in enumerate(iterator):
        texts_de = texts_de.T.to(DEVICE) # чтобы батч был в конце
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_en_input = texts_en[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        # подробнее про это по ссылкам выше
        (texts_de_mask, texts_en_mask, 
        texts_de_padding_mask, texts_en_padding_mask) = create_mask(texts_de, texts_en_input)
        logits = model(texts_de, texts_en_input, texts_de_mask, texts_en_mask,
                       texts_de_padding_mask, texts_en_padding_mask, texts_de_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_en_out = texts_en[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_en_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask, 
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)

            
            texts_ru_out = texts_ru[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [18]:
torch.manual_seed(0)

DE_VOCAB_SIZE = tokenizer_de.get_vocab_size()
EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, DE_VOCAB_SIZE, EN_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [19]:
torch.cuda.empty_cache()

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 100

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

#тут обучились на 4 эпохах где-то

In [22]:
def translate(text):


    input_ids = [tokenizer_de.token_to_id('[CLS]')] + tokenizer_de.encode(text).ids[:max_len_de] + [tokenizer_de.token_to_id('[SEP]')]
    output_ids = [tokenizer_en.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_de_mask, texts_en_mask, 
    texts_de_padding_mask, texts_en_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_de_mask, texts_en_mask,
                   texts_de_padding_mask, texts_en_padding_mask, texts_de_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_en.token_to_id('[SEP]'), tokenizer_en.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_de_mask, texts_en_mask, 
        texts_de_padding_mask, texts_en_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_de_mask, texts_en_mask,
                       texts_de_padding_mask, texts_en_padding_mask, texts_de_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_en.id_to_token(i).replace('##', '') for i in output_ids[1:]]))



In [25]:
translate("alles klar, Herr Komissar?")

'All right , Mr . Com iss ar ?'

In [24]:
for sent in de_sents[:100]:
    print('Deutsch: ', sent)
    print('English: ', translate(sent))
    print( )

#всё круто, я в восторге !! правда некоторый перевод неправильный, похож на гугл перевод из 2010

Deutsch:  Deine Habgier wird noch dein Tod sein.
English:  Your Majesty ' s gonna be your death .

Deutsch:  - Vega.
English:  - Ve ga .

Deutsch:  Sagen Sie einfach stopp.
English:  Just tell you ,

Deutsch:  - Warte.
English:  - Wait .

Deutsch:  Ich will nicht hier sein.
English:  I don ' t want to be here .

Deutsch:  Also, 90 Prozent meiner fotografischen Arbeit ist genau genommen gar nicht fotografisch.
English:  So , 90 % of my photo of my photo work is not photograph ed .

Deutsch:  Wie bezaubernd.
English:  Like a nice .

Deutsch:  Vielen Dank, Colonel.
English:  Thank you , Colonel .

Deutsch:  Martin!
English:  Martin !

Deutsch:  Muss ich?
English:  I ' m not a good thing .

Deutsch:  Nehmen Sie Platz.
English:  Take place .

Deutsch:  Leo!
English:  Leo !

Deutsch:  Nein, ehrlich gesagt finde ich, dass du herausfinden solltest, was mit deinem Dad los ist.
English:  No , I think I ' m sure you ' re going to figure out what ' s your dad ' s going on .

Deutsch:  Wie halten S


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Для обучения моделей машинного перевода необходимы параллельные корпуса, но их не так много, т.е. данных для обучения недостаточно. Поэтому можно взять монокорпус и просто добавить его к доступным параллельным корпусам. Эта техника заключается в том, чтобы создать битекст синтетически.
Сначала модель обучается на маленьком битексте, чтобы перевести данные на таргетном языке на язык-источник. Этот синтетически созданный битекст добавляется к данным для обучения.

То есть если хотим перевести текст с английского на русский, но у нас есть только маленький битекст, мы строим систему, чтобы перевести текст с русского на английский. Когда перевели текст на английский, то этот битекст теперь можно добавить к обучающим данным.